In [ ]:
from torch.optim.lr_scheduler import StepLR
import pandas as pd
import numpy as np
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset
import os
#import natsort
from torch.utils import data
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import random
from tqdm.auto import tqdm
seed = 12345
random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
train_annotations = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train_annotations.csv')


In [ ]:
train.head()

In [ ]:
# labels_dict = {train.iloc[i][0]:torch.Tensor(train.iloc[0][1:-1]) 
#               for i in range(len(train))}

In [ ]:
labels_dict = {}
for i in range(len(train)):
    k = train.iloc[i][0]
    v = torch.Tensor(train.iloc[0][1:-1])
    labels_dict[k] = v

In [ ]:
labels_dict['1.2.826.0.1.3680043.8.498.10000428974990117276582711948006105617']

In [ ]:
#the commented things here can change the images to become 3 channel
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform, labels):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = all_imgs#natsort.natsorted(all_imgs)
        self.labels = labels

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = Image.open(img_loc)#.convert("RGB")
        tensor_image = self.transform(image)
        key = os.path.basename(img_loc)[:-4]#this is how we get our ids
        #input labels as dictionary with id number as the key
        label = self.labels[key]
        return tensor_image, label

train_transform = transforms.Compose([
     transforms.Resize((64,64)),
     transforms.ToTensor(),
     transforms.Normalize(
         [0.4826],# 0.4824, 0.4824],
         [0.2190]),# 0.2142, 0.2142])
 ])

In [ ]:
images = CustomDataSet('../input/ranzcr-clip-catheter-line-classification/train',train_transform, labels_dict)

In [ ]:
len(images)

In [ ]:
random_seed = 12345
validation_split = .2
shuffle_dataset = True

dataset_size = len(images)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]


In [ ]:
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

In [ ]:
batch_size = 256
train_loader = torch.utils.data.DataLoader(images, batch_size=batch_size, 
                                           sampler=train_sampler, num_workers = 4)
val_loader = torch.utils.data.DataLoader(images, batch_size=batch_size,
                                                sampler=val_sampler, num_workers = 4)


In [ ]:
class View(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape

    def forward(self, x):
        return x.view(*self.shape)
    
    
    
class ShallowConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        """

        Parameters
        ----------
        input_channels : Number of input channels
        num_classes : Number of classes for the final prediction 
        """
        
        super().__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes
        
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        
        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 8)
        )
        
        self.fc = nn.Linear(256, self.num_classes)
        self.sig = nn.Sigmoid()


    def forward(self, x):
        
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)
            
        x = View((-1,256))(x)
            
        x = self.fc(x)
        x = self.sig(x)
        return x



        
    
class SimpleConvnet(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(SimpleConvnet, self).__init__()

        self.input_channels = input_channels
        self.num_classes = num_classes

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels = self.input_channels, out_channels = 64, kernel_size=5, padding=2),
            nn.ReLU(),
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size = 2)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size = 16)
        )

        #final linear layer to project into the correct number of classes
        self.fc = nn.Linear(256, 11)
        self.sig = nn.Sigmoid()
    
    def forward(self, x):
       
        x = self.block1(x)

        x = self.block2(x)

        x = self.block3(x)

        x = self.block4(x)

        x = View((-1,256))(x)
        x = self.fc(x)
        x = self.sig(x)
        output = x
        
        return output
    
    

In [ ]:
def train_loop(model, criterion, optimizer,  train_loader, val_loader):
    """
    Generic training loop

    Parameters
    ----------
    model : Object instance of your model class 
    criterion : Loss function 
    optimizer : Instance of optimizer class of your choice 
    train_loader : Training data loader 
    val_loader : Validation data loader

    Returns
    -------
    train_losses : List with train loss on dataset per epoch
    train_accuracies : List with train accuracy on dataset per epoch
    val_losses : List with validation loss on dataset per epoch
    val_accuracies : List with validation accuracy on dataset per epoch

    """
    best_val = 0.0
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    max_patience = 5
    patience_counter = 0
    # Training
    for t in tqdm(range(50)):
        epoch_t_acc = 0.0 
        epoch_t_loss = 0.0
        model.train()       
        for i, samples in enumerate(train_loader):
            data, target = samples
            target = target.long()
            data, target = data.to(device), target.to(device)
            
            y_pred_train = model(data)
            #y_pred_train = y_pred_train.round()

            loss = criterion(y_pred_train, target)
#             score, predicted = torch.max(y_pred_train, 1)
#             acc = (predicted == target).sum().float() / len(target)
            # for each example in the batch
            each_ex_acc = (target == y_pred_train.round()).sum(dim=1)/len(target[0])
            acc = each_ex_acc.sum()/len(target)
            
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_t_acc += acc
            epoch_t_loss += loss.item()

        train_accuracies.append(epoch_t_acc/len(train_loader))
        train_losses.append(epoch_t_loss/len(train_loader))

        model.eval()

        v_acc = 0.0
        v_loss = 0.0


        with torch.no_grad():
        # TODO: Loop over the validation set 
            for i, samples in enumerate(val_loader):

                # TODO: Put the inputs and targets on the write device
                data, target = samples
                target = target.long()
                data, target = data.to(device), target.to(device)

                # TODO: Feed forward to get the logits
                y_pred_val = model(data)

                # TODO: Compute the loss and accuracy
                loss = criterion(y_pred_val, target)
#                 score, predicted = torch.max(y_pred_val, 1)
#                 acc = (predicted == target).sum().float() / len(target)
                each_ex_acc = (target == y_pred_val.round()).sum(dim=1)/len(target[0])
                acc = each_ex_acc.sum()/len(target)
        
                v_loss+= loss.item()
                v_acc += acc


        # TODO: Keep track of accuracy and loss
        val_accuracies.append(v_acc/len(val_loader))
        val_losses.append(v_loss/len(val_loader))

        if val_accuracies[-1] > best_val:
            best_val = val_accuracies[-1]
            patience_counter = 0

      # TODO: Save best model, optimizer, epoch_number
    
    
    
            torch.save(model.state_dict(), './model_state.pt')

        else:
            patience_counter += 1    
            if patience_counter > max_patience: 
                break

        print("[EPOCH]: %i, [TRAIN LOSS]: %.6f, [TRAIN ACCURACY]: %.5f" % (t, train_losses[-1], train_accuracies[-1]))
        print("[EPOCH]: %i, [VAL LOSS]: %.6f, [VAL ACCURACY]: %.5f \n" % (t, val_losses[-1] ,val_accuracies[-1]))

    return train_losses, train_accuracies, val_losses, val_accuracies

In [ ]:
# TODO : Initialize the model and cast to correct device
input_channels = 1
num_classes = 11
model_sc = SimpleConvnet(input_channels, num_classes)
model_sc.to(device)

# TODO : Initialize the criterion
criterion = torch.nn.MultiLabelSoftMarginLoss()
# TODO : Initialize the SGD optimizer with lr 1e-3
optimizer = optim.SGD(model_sc.parameters(), lr = 0.001)

# TODO : Run the training loop using this model

train_losses, train_accuracies, val_losses, val_accuracies = train_loop(model_sc, criterion, optimizer, train_loader, val_loader)




In [ ]:
mordel = torch.load('./model_state.pt')

In [ ]:
mogle = SimpleConvnet(1,11)
mogle.load_state_dict(mordel)

In [ ]:
mordel

In [ ]:
nimages = 0
mean = 0.
std = 0.
for batch, _ in train_loader:
    # Rearrange batch to be the shape of [B, C, W * H]
    batch = batch.view(batch.size(0), batch.size(1), -1)
    # Update total number of images
    nimages += batch.size(0)
    # Compute mean and std here
    mean += batch.mean(2).sum(0) 
    std += batch.std(2).sum(0)

# Final step
mean /= nimages
std /= nimages

print(mean)
print(std)